<a href="https://colab.research.google.com/github/naokityokoyama/infnet/blob/main/infnet_dash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install dash
!pip install dash_bootstrap_components
!pip install jupyter-dash

     |████████████████████████████████| 81kB 3.4MB/s 
     |████████████████████████████████| 1.0MB 11.3MB/s 
     |████████████████████████████████| 3.5MB 28.7MB/s 
     |████████████████████████████████| 92kB 7.8MB/s 
     |████████████████████████████████| 1.8MB 29.2MB/s 
     |████████████████████████████████| 358kB 35.9MB/s 
  Created wheel for dash: filename=dash-1.20.0-cp37-none-any.whl size=85845 sha256=6311daf669a075e07b49d151cd5569c0d433f06014302bf29ad9a2c3b61e6fae
  Stored in directory: /root/.cache/pip/wheels/e6/33/41/ce361d0d1da277d2350f815b10f2ab1e331592d93ef2b07e29
  Created wheel for dash-renderer: filename=dash_renderer-1.9.1-cp37-none-any.whl size=1014873 sha256=fd01becb60c9baec463d72757ac0ae97bb81f278726e7e5346601225689645b0
  Stored in directory: /root/.cache/pip/wheels/03/a9/c5/dd5815c601b0ede164c223ffd7bafebde716ca57de06ef8aec
  Created wheel for dash-core-components: filename=dash_core_components-1.16.0-cp37-none-any.whl size=3540992 sha256=3e82e1fe68c29baadfb21a

In [ ]:
from fbprophet import Prophet
import pandas as pd
import plotly.express as px
from jupyter_dash import JupyterDash

import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
import plotly.express as px 
from dash.dependencies import Input, Output

#dataset
df = pd.read_csv('https://raw.githubusercontent.com/naokityokoyama/infnet/main/train.csv')

lista_lojas = range(1,46)
lista_semanas = range(2,6)

def select_store_1(n,p):
  data = df[df['Store']==n]
  data = data.drop(['Dept', 'IsHoliday', 'Store'], axis=1)
  data = data.rename(columns={'Date':'ds', 'Weekly_Sales':'y'})
  data = data.groupby('ds').sum().reset_index()

  m = Prophet(yearly_seasonality=True)
  m.add_country_holidays(country_name='US')
  m.fit(data)
  future = m.make_future_dataframe(periods=p, freq='w')
  forecast = m.predict(future)
  return round(forecast[['ds', 'yhat']]).tail(p)

def select_store_2(n,p):
  data = df[df['Store']==n]
  data = data.drop(['Dept', 'IsHoliday', 'Store'], axis=1)
  data = data.rename(columns={'Date':'ds', 'Weekly_Sales':'y'})
  data = data.groupby('ds').sum().reset_index()

  m = Prophet(yearly_seasonality=True)
  m.add_country_holidays(country_name='US')
  m.fit(data)
  future = m.make_future_dataframe(periods=p, freq='w')
  forecast = m.predict(future)
  return round(forecast[['ds', 'yhat']])

app = JupyterDash(external_stylesheets=[dbc.themes.CYBORG])

controls = dbc.Card(
    [
        dbc.FormGroup(
            [
                dbc.Label("Select Store"),
                dcc.Dropdown(
                    id="x-variable",
                    value='1',
                    options=[
                        {"label": col, "value": col} for col in lista_lojas
                    ]
                 
                )
            ]
        ),
        dbc.FormGroup(
            [
                dbc.Label("Select week number"),
                dcc.Dropdown(
                    id="y-variable",
                    value='2',
                    options=[
                        {"label": col, "value": col} for col in lista_semanas
                    ]
                    
                )
            ]
        )
      ],
    #body=True,
)

grafico1 = dcc.Graph(id='grafico1', config={'displayModeBar':False})
grafico2 = dcc.Graph(id='grafico2', config={'displayModeBar':False})


grid1 = html.Div([
                  
    #dbc.Row(dbc.Col(dbc.Alert('Barra de inicio', color='primary')))
#])

    dbc.Alert(
            [
                html.H4("Dash Wallmart"),
                #html.A("examplo link", href="#", className="alert-heading"),
                html.Img(src='https://www.feltex.com.br/felix/wp-content/uploads/2014/03/Infnet-Logo.png',
                         style={
                'maxWidth': '10%',
                'maxHeight': '10%',
                'marginLeft': '900px',
                'marginRight': 'auto',
                'marginUp': '1%'},
                )
            ]
           , color="primary",
     )])

grid2 = html.Div([
    dbc.Row([
        dbc.Col(controls, md=2),
        dbc.Col(grafico1, md=4),
        dbc.Col(grafico2, md=6)
     ], align='center')
])

app.layout = html.Div([
    grid1, grid2
    ])

#callback Grafico 1
@app.callback(
    Output('grafico1', 'figure'),
    [Input('x-variable', 'value'),
     Input('y-variable', 'value')]
)

#Grafico 1
def criando_grafico(x,y):
    
    filtro_df = select_store_1(x,y)
    fig = px.line(filtro_df,  x='ds', y='yhat', title=f"Serie week {y} of store {x}", labels={'ds':'Date', 'yhat':'Sales'})
    fig.update_layout({'paper_bgcolor':'rgb(28,28,28)'})
    fig.update_layout(font=dict(
        family="Courier New, monospace",
        size=14,
        color="White"))
    return fig

#callback Grafico 2
@app.callback(
    Output('grafico2', 'figure'),
    [Input('x-variable', 'value'),
     Input('y-variable', 'value')]
)

#grafico 2
def criando_grafico(x,y):
    
    filtro_df = select_store_2(x,y)
    fig = px.line(filtro_df,  x='ds', y='yhat', title=f"series history of store {x} ", labels={'ds':'Date', 'yhat':'Sales'})
    fig.update_layout({'paper_bgcolor':'rgb(28,28,28)'})
    fig.update_layout(font=dict(
        family="Courier New, monospace",
        size=14,
        color="White"))
    return fig    

if __name__ == '__main__':
  app.run_server(mode='inline')

<IPython.core.display.Javascript object>